In [7]:
import numpy as np
import pandas as pd
from ast import literal_eval
from collections import Counter
import tensorflow as tf

Let's deal at first with Common Nouns Question Answering.

In [2]:
raw_train = pd.read_csv('data/CBT_CN_train.csv',delimiter=';')
raw_valid = pd.read_csv('data/CBT_CN_valid.csv',delimiter=';')

In [4]:
raw_train = raw_train.sample(n=15000, replace=True)

In [5]:
raw_train['Document'] = raw_train['Document'].apply(literal_eval)
raw_train['Query'] = raw_train['Query'].apply(literal_eval)
raw_train['Candidates'] = raw_train['Candidates'].apply(literal_eval)

In [6]:
raw_train.head()

,Document,Query,Candidates,Answer
676,"[the, king, was, so, surprised, and, hurt, at,...","[now, ,, as, pantouflia, was, a, rich, ,, lazy...","[accent, beds, bit, country, earth, fairy, for...",country
60365,"[all, of, this, will, be, expensive, ,, amy, ....","[you, can, go, through, the, xxxxx, with, your...","[talk, climax, cost, girls, heart, mood, novel...",world
119215,"[that, s, only, bowser, the, hound, chasing, o...","[they, do, n, t, seem, to, think, anything, ab...","[forest, hound, boy, care, gun, hands, hunt, p...",gun
86768,"[where, do, they, build, their, nests, ,, and,...","[might, not, ,, then, ,, its, more, appropriat...","[affairs, crowd, dwelling, eve, food, passage,...",town
37636,"[all, right, ,, said, she, ,, and, held, a, bi...","[but, the, ogre, s, xxxxx, was, still, alive, ...","[chance, death, door, fish, house, men, relati...",wife


### 1. Maximum frequency model.

In [21]:
# choose the candidate that has the maximum frequency in Query/Document
def maximum_frequency_prediction(row, use_Document_info=False):
    temp = row['Query']
    if use_Document_info:
        temp += row['Document']
    freqs = dict(Counter(temp))
    ans = row['Candidates'][0]
    for w in row['Candidates']:
        if w in freqs.keys() and (ans not in freqs.keys() or (freqs[w] > freqs[ans])):
            ans = w
    return ans

In [32]:
def max_freq_result(df, use_Document_info=False):
    accuracy = 0
    for _, row in df.iterrows():
        if maximum_frequency_prediction(row, use_Document_info) == row['Answer']:
            accuracy += 1
    print('\t Accuracy:', round(accuracy / raw_train.shape[0], 2))

In [33]:
print('Maximum frequency (corpus):')
max_freq_result(raw_train, False)
print('Maximum frequency (corpus + context):')
max_freq_result(raw_train, True)

Maximum frequency (corpus):
	 Accuracy: 0.13
Maximum frequency (corpus + context):
	 Accuracy: 0.27


In the paper "THE GOLDILOCKS PRINCIPLE: READING CHILDREN’S BOOKS..." the results are $0.158$ with corpus-only and $0.281$ with corpus+context, which is pretty similar.